# PCA coding exercise 1 solution: visualizing PCA

In [1]:
import numpy as np

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

Loading BokehJS ...

# Your dataset

In [2]:
np.random.seed(0)

x = np.random.normal(0,10,150)
y = x + np.random.normal(0,5,150)

In [3]:
data = np.vstack((x,y)).T

## Look at the data

In [4]:
r = 35
p = bokeh.plotting.figure(width = 550, height = 300,
                         title = 'Input data',
                         y_range = (-r,r),
                         x_range = (-r,r),
                         # aspect_ratio = 1
                          y_axis_label = 'y',
                          x_axis_label = 'x',
                         )

p.circle(data[:,0],data[:,1], color = '#CE8964',
        legend_label = 'raw data')



bokeh.io.show(p)

## Mean center the data

Let the features = 2, observations = 100. Then, we will mean center each feature over all observations.

In [5]:
data_centered = data - np.mean(data, axis=0)

Look at the data over the raw data

In [6]:
p.circle(data_centered[:,0],data_centered[:,1], color = '#843B62',
        legend_label = 'mean centered data')
p.add_layout(p.legend[0], 'right')
bokeh.io.show(p)

## Calculate the covariance matrix, eigenvalues, eigenvectors

In [7]:
cov_matrix = np.cov(data_centered, rowvar=False, bias = False)

In [8]:
evals, evecs = np.linalg.eig(cov_matrix)

sorted_eig = sorted(zip(evals, evecs))
sorted_eig

[(11.362799813713764, array([-0.74100028, -0.67150472])),
 (218.55966043224902, array([ 0.67150472, -0.74100028]))]

## Plot the principal components

In [9]:
colors = ['#843B62', '#326273']

r = 35
p1 = bokeh.plotting.figure(width = 500, height = 350,
                           title = 'Data with PCs',
                           x_axis_label = 'x',
                           y_axis_label = 'y',
                          x_range = (-r, r),
                           y_range = (-r,r)
                          )

for i,pc in enumerate(sorted_eig):
    curr_evec = sorted_eig[i][1]
    curr_eval = sorted_eig[i][0]
    # plot the lines so it covers entire plot (x10)
    endpt = curr_evec * curr_eval * 100
    
    # plot the PC lines
    p1.line(x = (-endpt[0], endpt[0]),
            y = (-endpt[1], endpt[1]),
                 color = colors[i],
                 legend_label = 'PC' + str(i+1),
                 width = 3)

# plot the data
p1.circle(x,y, color = '#CE8964', legend_label = 'data')
p1.add_layout(p1.legend[0], 'right')

bokeh.io.show(p1)

We can see that PC1 cuts through the main axes of the data.

Show that the PC's are orthogonal by taking their dot product

In [10]:
np.dot(sorted_eig[0][1], sorted_eig[1][1])

0.0

## Rotate the data

Define a function to help rotate the data

In [11]:
def rotate_func(x,y):
    evec1 = sorted_eig[0][1]
    evec2 = sorted_eig[1][1]
    
    new_x = -(evec1[0] * x + evec1[1] * y)
    new_y = -(evec2[0] * x + evec2[1] * y)
    
    return new_x, new_y

In [12]:
newx = []
newy = []

for curr_data in data_centered:
    new_x, new_y = rotate_func(curr_data[0], curr_data[1])
    
    newx.append(new_x)
    newy.append(new_y)
#ewx

In [13]:
r = 30
p2 = bokeh.plotting.figure(width = 350, height = 300,
                         #title = 'example data with PC axes',
                         y_range = (-r,r),
                         x_range = (-r,r),
                         # aspect_ratio = 1
                          y_axis_label = 'PC2',
                          x_axis_label = 'PC1'
                         )
p2.circle(newx, newy, color = '#CE8964')
bokeh.io.show(p2)

## Proportion of variance

In [15]:
eval1 = sorted_eig[0][0]
eval2 = sorted_eig[1][0]
total = eval1 + eval2

In [16]:
print('eigenvalue 1 represents', eval1/total, 'of the variance')
print('eigenvalue 2 represents', eval2/total, 'of the variance')

eigenvalue 1 represents 0.04942013843083555 of the variance
eigenvalue 2 represents 0.9505798615691644 of the variance


## L vs M plot

In [17]:
data.shape

(150, 2)

In [18]:
M = data.shape[0]
L = data.shape[1]

In [19]:
x_lm = np.linspace(0,M + 10,100)
y_lm1 = x_lm * eval1
y_lm2 = x_lm * eval2


p3 = bokeh.plotting.figure(width = 400, height = 300,
                          y_axis_label = 'L',
                          x_axis_label = 'M',
                          title = '(M,L) = (100,2)')

p3.line(x = x_lm, y = y_lm1, legend_label = 'r1', color = colors[0], line_width = 3)
p3.line(x = x_lm, y = y_lm2, legend_label = 'r2', color = colors[1], line_width = 3)
p3.circle(x = M, y = L, size = 10, color = '#CE8964', legend_label = '(M,L) coordinate')
p3.legend.location = 'top_left'

bokeh.io.show(p3)

From this plot, we see that we are in the regime where we might be able to trust PC1 and PC2.

In [20]:
%load_ext watermark
%watermark -v -p numpy,bokeh,jupyterlab

Python implementation: CPython
Python version       : 3.9.13
IPython version      : 7.31.1

numpy     : 1.21.6
bokeh     : 3.2.0
jupyterlab: 3.4.4

